In [1]:

from bmapqml.utils import analyze_random_walk
import numpy as np

from bmapqml.chemxpl.minimized_functions import QM9_properties,multi_obj,Rdkit_properties
path = "/home/jan/projects/MOLOPT/do_sim/bias/both/9/"
ana = analyze_random_walk("{}".format(path)+"QM9_histogram.pkl","{}".format(path)+"QM9_best_candidates.pkl" )

WEIGHTS = np.array([ (1/1.9), (1/6.8)])

min_func = multi_obj(
    [QM9_properties("/store/common/jan/qm9/KRR_12000_atomization",verbose=False),QM9_properties("/store/common/jan/qm9/KRR_12000_gap",verbose=False)
], WEIGHTS, verbose=True)


[18:55:59] Enabling RDKit 2019.09.3 jupyter extensions
RDKit WARNING: [18:55:59] Enabling RDKit 2019.09.3 jupyter extensions


In [5]:
min_func.__call__(ana.tps[1])

SMILE: CCCCCCCCN v1 -1.9783090435477597 v2 8.651974443278005


0.23113389675723273

In [9]:
ana.saved_candidates_func_val

array([-0.68427093, -0.68290036, -0.67865705, -0.66244964])